<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [5]:
!pip install accelerate
!huggingface-cli login
!pip install datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [6]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

#**loading the model**

In [7]:
# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3.1-8B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("How old are you?"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'How old are you? I am 34.\nWhat’s your favorite song? “The Boxer” by Simon and Gar'}]


# helper functions

In [8]:
def generate_answer(question, num_samples):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Generate all 10 answers in a single call
    outputs = pipeline(
        prompt,
        max_length=100,
        truncation=True,
        temperature=0.5,  # Fixed temperature for all answers
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples
    )

    # Extract generated answers
    responses = [
        output['generated_text'].split("Answer:")[-1].strip() for output in outputs
    ]

    return responses

# main function to to load questions and gerate answers and save them to a file

In [13]:
# Main function
def main():

    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:
      # Load the XQuAD dataset for Arabic (ar)
      xquad_dataset = load_dataset("xquad", "xquad.ar")

      # Load the validation set (the only available split)
      test_set = xquad_dataset['validation']


      # Create an empty list to store the data
      data = []

      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['answers']


          # Append the data as a dictionary to the list
          data.append({
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })

    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=100).iterrows() :
        question = row['Question']
        original_answer = row.get('Answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)
#change the file name !!!!!!!!!!!!!!!!!!!
    filename = "generated_answers_Llama3.1-xquad.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    files.download(filename)

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

    question_id                                           question  \
0           320                             ما هو المرفق بالسواكن؟   
1          1007     ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟   
2           605  ماذا حدث في عام 1992 ضمن إجراءٍ على نطاق الممل...   
3           425                   متى أحدِثت توجيهية مجلس الأعمال؟   
4             4  من سجل أعلى عدد من الاستحواذات للفريق هذا الموسم؟   
..          ...                                                ...   
95          778  ‏ما الذي كان من المرجح أن يكون عليه المهندسون ...   
96         1082             متى تم تمرير قانون الجنسية البريطانية؟   
97          506                          كم كان متوسط حجم العائلة؟   
98          908  ما هي الإدارة التي صمَّمَ لها لودفيج ميس فان د...   
99          146  تصنف نظرية التعقيد المسائل بناء على أي سمة أسا...   

                                      original_answer  \
0   {'text': ['غلاف التوربين'], 'answer_start': [1...   
1   {'text': ['وجود البلاستيدات الخضراء المفق

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>